## Importing Pandas and creating a list `tickers` to load data of the companies required

In [3]:
import pandas as pd

tickers = ['aapl', 'msft', 'amzn', 'tsla', 'googl']
dfs = []

## Appending each dataframe to `dfs` each at a time and concatenating all of them at the end

In [4]:
for ticker in tickers:
    df = pd.read_csv(f"./Stocks/{ticker}.us.txt", delimiter=',')
    df.insert(0, 'Ticker', ticker)
    dfs.append(df)

df = pd.concat(dfs)

## Converting `Dates` to `datetime` format 

In [5]:
df['Date'] = pd.to_datetime(df['Date'])

## Setting `Ticker` and `Date` as Multi-level Index

In [6]:
df.set_index(['Ticker', 'Date'], inplace=True)

In [7]:
print(df)

                         Open        High         Low       Close    Volume  \
Ticker Date                                                                   
aapl   1984-09-07     0.42388     0.42902     0.41874     0.42388  23220030   
       1984-09-10     0.42388     0.42516     0.41366     0.42134  18022532   
       1984-09-11     0.42516     0.43668     0.42516     0.42902  42498199   
       1984-09-12     0.42902     0.43157     0.41618     0.41618  37125801   
       1984-09-13     0.43927     0.44052     0.43927     0.43927  57822062   
...                       ...         ...         ...         ...       ...   
googl  2017-11-06  1049.10000  1052.59000  1042.00000  1042.68000    913954   
       2017-11-07  1049.65000  1053.41000  1043.00000  1052.39000   1303832   
       2017-11-08  1050.05000  1062.69000  1047.05000  1058.29000   1214469   
       2017-11-09  1048.00000  1050.88000  1035.85000  1047.72000   1793994   
       2017-11-10  1043.87000  1046.63000  1041.2200

## Sorting the `Date`

In [8]:
df.sort_index(level=['Ticker', 'Date'], inplace=True)
print(df)

                        Open       High        Low      Close    Volume  \
Ticker Date                                                               
aapl   1984-09-07    0.42388    0.42902    0.41874    0.42388  23220030   
       1984-09-10    0.42388    0.42516    0.41366    0.42134  18022532   
       1984-09-11    0.42516    0.43668    0.42516    0.42902  42498199   
       1984-09-12    0.42902    0.43157    0.41618    0.41618  37125801   
       1984-09-13    0.43927    0.44052    0.43927    0.43927  57822062   
...                      ...        ...        ...        ...       ...   
tsla   2017-11-06  307.00000  307.50000  299.01000  302.78000   6482486   
       2017-11-07  301.02000  306.50000  300.03000  306.05000   5286320   
       2017-11-08  305.50000  306.89000  301.30000  304.31000   4725510   
       2017-11-09  302.50000  304.46000  296.30000  302.99000   5440335   
       2017-11-10  302.50000  308.36000  301.85000  302.99000   4621912   

                   OpenI

## Identifying missing rows per `Ticker`

In [9]:
missing_per_ticker = df.groupby(level='Ticker').apply(lambda x: x.isna().sum())
print(missing_per_ticker)

        Open  High  Low  Close  Volume  OpenInt
Ticker                                         
aapl       0     0    0      0       0        0
amzn       0     0    0      0       0        0
googl      0     0    0      0       0        0
msft       0     0    0      0       0        0
tsla       0     0    0      0       0        0


## Using `ffill` and `bfill` for missing data

In [10]:
df = df.groupby(level='Ticker').ffill()
df = df.groupby(level='Ticker').bfill()

## Filtering the data to include only 10 years (2007-2017)

In [11]:
time = (df.index.get_level_values('Date') >= '2007-01-01') & (df.index.get_level_values('Date') <= '2018-01-01')
filtered_df = df[time]
print(filtered_df)

                      Open     High      Low    Close     Volume  OpenInt
Ticker Date                                                              
aapl   2007-01-03   11.049   11.087   10.486   10.731  345302870        0
       2007-01-04   10.780   11.007   10.733   10.969  236252357        0
       2007-01-05   10.992   11.038   10.807   10.893  232773093        0
       2007-01-08   11.011   11.081   10.922   10.946  222149027        0
       2007-01-09   11.075   11.907   10.906   11.854  933759387        0
...                    ...      ...      ...      ...        ...      ...
tsla   2017-11-06  307.000  307.500  299.010  302.780    6482486        0
       2017-11-07  301.020  306.500  300.030  306.050    5286320        0
       2017-11-08  305.500  306.890  301.300  304.310    4725510        0
       2017-11-09  302.500  304.460  296.300  302.990    5440335        0
       2017-11-10  302.500  308.360  301.850  302.990    4621912        0

[12802 rows x 6 columns]


## Calcuting `Daily Return` (% change in closing price)

In [12]:
filtered_df['Daily Return'] = filtered_df.groupby(level='Ticker')['Close'].pct_change()

C:\Users\vines\AppData\Local\Temp\ipykernel_7256\444591979.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Daily Return'] = filtered_df.groupby(level='Ticker')['Close'].pct_change()


In [13]:
filtered_df

Open     High      Low    Close     Volume  OpenInt  \
Ticker Date                                                                 
aapl   2007-01-03   11.049   11.087   10.486   10.731  345302870        0   
       2007-01-04   10.780   11.007   10.733   10.969  236252357        0   
       2007-01-05   10.992   11.038   10.807   10.893  232773093        0   
       2007-01-08   11.011   11.081   10.922   10.946  222149027        0   
       2007-01-09   11.075   11.907   10.906   11.854  933759387        0   
...                    ...      ...      ...      ...        ...      ...   
tsla   2017-11-06  307.000  307.500  299.010  302.780    6482486        0   
       2017-11-07  301.020  306.500  300.030  306.050    5286320        0   
       2017-11-08  305.500  306.890  301.300  304.310    4725510        0   
       2017-11-09  302.500  304.460  296.300  302.990    5440335        0   
       2017-11-10  302.500  308.360  301.850  302.990    4621912        0   

                   Daily Return  
Ticker Date                      
aapl   2007-01-03           NaN  
       2007-01-04      0.022179  
       2007-01-05     -0.006929  
       2007-01-08      0.004866  
       2007-01-09      0.082953  
...                         ...  
tsla   2017-11-06     -0.010814  
       2017-11-07      0.010800  
       2017-11-08     -0.005685  
       2017-11-09     -0.004338  
       2017-11-10      0.000000  

[12802 rows x 7 columns]

## Calculating `7-day Moving Average` of closing price

In [14]:
filtered_df['7-day Moving Average'] = filtered_df['Close'].rolling(7).mean()

C:\Users\vines\AppData\Local\Temp\ipykernel_7256\3533965052.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['7-day Moving Average'] = filtered_df['Close'].rolling(7).mean()


In [15]:
filtered_df

Open     High      Low    Close     Volume  OpenInt  \
Ticker Date                                                                 
aapl   2007-01-03   11.049   11.087   10.486   10.731  345302870        0   
       2007-01-04   10.780   11.007   10.733   10.969  236252357        0   
       2007-01-05   10.992   11.038   10.807   10.893  232773093        0   
       2007-01-08   11.011   11.081   10.922   10.946  222149027        0   
       2007-01-09   11.075   11.907   10.906   11.854  933759387        0   
...                    ...      ...      ...      ...        ...      ...   
tsla   2017-11-06  307.000  307.500  299.010  302.780    6482486        0   
       2017-11-07  301.020  306.500  300.030  306.050    5286320        0   
       2017-11-08  305.500  306.890  301.300  304.310    4725510        0   
       2017-11-09  302.500  304.460  296.300  302.990    5440335        0   
       2017-11-10  302.500  308.360  301.850  302.990    4621912        0   

                   Daily Return  7-day Moving Average  
Ticker Date                                            
aapl   2007-01-03           NaN                   NaN  
       2007-01-04      0.022179                   NaN  
       2007-01-05     -0.006929                   NaN  
       2007-01-08      0.004866                   NaN  
       2007-01-09      0.082953                   NaN  
...                         ...                   ...  
tsla   2017-11-06     -0.010814            314.527143  
       2017-11-07      0.010800            312.410000  
       2017-11-08     -0.005685            310.157143  
       2017-11-09     -0.004338            306.080000  
       2017-11-10      0.000000            303.495714  

[12802 rows x 8 columns]

## Calculating `30-day Moving Average` of closing price

In [16]:
filtered_df['30-day Moving Average'] = filtered_df['Close'].rolling(30).mean()

C:\Users\vines\AppData\Local\Temp\ipykernel_7256\1839080862.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['30-day Moving Average'] = filtered_df['Close'].rolling(30).mean()


In [17]:
filtered_df

Open     High      Low    Close     Volume  OpenInt  \
Ticker Date                                                                 
aapl   2007-01-03   11.049   11.087   10.486   10.731  345302870        0   
       2007-01-04   10.780   11.007   10.733   10.969  236252357        0   
       2007-01-05   10.992   11.038   10.807   10.893  232773093        0   
       2007-01-08   11.011   11.081   10.922   10.946  222149027        0   
       2007-01-09   11.075   11.907   10.906   11.854  933759387        0   
...                    ...      ...      ...      ...        ...      ...   
tsla   2017-11-06  307.000  307.500  299.010  302.780    6482486        0   
       2017-11-07  301.020  306.500  300.030  306.050    5286320        0   
       2017-11-08  305.500  306.890  301.300  304.310    4725510        0   
       2017-11-09  302.500  304.460  296.300  302.990    5440335        0   
       2017-11-10  302.500  308.360  301.850  302.990    4621912        0   

                   Daily Return  7-day Moving Average  30-day Moving Average  
Ticker Date                                                                   
aapl   2007-01-03           NaN                   NaN                    NaN  
       2007-01-04      0.022179                   NaN                    NaN  
       2007-01-05     -0.006929                   NaN                    NaN  
       2007-01-08      0.004866                   NaN                    NaN  
       2007-01-09      0.082953                   NaN                    NaN  
...                         ...                   ...                    ...  
tsla   2017-11-06     -0.010814            314.527143             339.294800  
       2017-11-07      0.010800            312.410000             337.988133  
       2017-11-08     -0.005685            310.157143             336.766133  
       2017-11-09     -0.004338            306.080000             335.545800  
       2017-11-10      0.000000            303.495714             334.278133  

[12802 rows x 9 columns]

## Calculating `Rolling Volatility (30d)` of closing price

In [18]:
filtered_df['Rolling Volatility (30d)'] = filtered_df['Daily Return'].rolling(30).std()

C:\Users\vines\AppData\Local\Temp\ipykernel_7256\2393570958.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Rolling Volatility (30d)'] = filtered_df['Daily Return'].rolling(30).std()


In [19]:
filtered_df

Open     High      Low    Close     Volume  OpenInt  \
Ticker Date                                                                 
aapl   2007-01-03   11.049   11.087   10.486   10.731  345302870        0   
       2007-01-04   10.780   11.007   10.733   10.969  236252357        0   
       2007-01-05   10.992   11.038   10.807   10.893  232773093        0   
       2007-01-08   11.011   11.081   10.922   10.946  222149027        0   
       2007-01-09   11.075   11.907   10.906   11.854  933759387        0   
...                    ...      ...      ...      ...        ...      ...   
tsla   2017-11-06  307.000  307.500  299.010  302.780    6482486        0   
       2017-11-07  301.020  306.500  300.030  306.050    5286320        0   
       2017-11-08  305.500  306.890  301.300  304.310    4725510        0   
       2017-11-09  302.500  304.460  296.300  302.990    5440335        0   
       2017-11-10  302.500  308.360  301.850  302.990    4621912        0   

                   Daily Return  7-day Moving Average  30-day Moving Average  \
Ticker Date                                                                    
aapl   2007-01-03           NaN                   NaN                    NaN   
       2007-01-04      0.022179                   NaN                    NaN   
       2007-01-05     -0.006929                   NaN                    NaN   
       2007-01-08      0.004866                   NaN                    NaN   
       2007-01-09      0.082953                   NaN                    NaN   
...                         ...                   ...                    ...   
tsla   2017-11-06     -0.010814            314.527143             339.294800   
       2017-11-07      0.010800            312.410000             337.988133   
       2017-11-08     -0.005685            310.157143             336.766133   
       2017-11-09     -0.004338            306.080000             335.545800   
       2017-11-10      0.000000            303.495714             334.278133   

                   Rolling Volatility (30d)  
Ticker Date                                  
aapl   2007-01-03                       NaN  
       2007-01-04                       NaN  
       2007-01-05                       NaN  
       2007-01-08                       NaN  
       2007-01-09                       NaN  
...                                     ...  
tsla   2017-11-06                  0.022166  
       2017-11-07                  0.022317  
       2017-11-08                  0.022261  
       2017-11-09                  0.022261  
       2017-11-10                  0.022224  

[12802 rows x 10 columns]

## Calculating Average Returns grouped by `Ticker`

In [20]:
avgr = filtered_df.groupby(level='Ticker')['Daily Return'].mean()

## Obtaining the `Ticker` and `highest_avg_return`

In [21]:
best_ticker = avgr.idxmax()
highest_avg_return = avgr.max()

print(f"Ticker with highest average return: {best_ticker} ({highest_avg_return:.2%})")

Ticker with highest average return: tsla (0.21%)


## Finding the `stock` which had the most volatile month and the month of appearance

In [22]:
filtered_df['Daily_Return'] = filtered_df.groupby(level='Ticker')['Close'].pct_change()
filtered_df_reset = filtered_df.reset_index()
filtered_df_reset['Year_Month'] = filtered_df_reset['Date'].dt.to_period('M')
monthly_volatility = filtered_df_reset.groupby(['Ticker', 'Year_Month'])['Daily_Return'].std()

C:\Users\vines\AppData\Local\Temp\ipykernel_7256\1720986004.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Daily_Return'] = filtered_df.groupby(level='Ticker')['Close'].pct_change()


## The final result

In [23]:
most_volatile = monthly_volatility.idxmax()
highest_volatility = monthly_volatility.max()
print(f"Most volatile month: {most_volatile[1]} for {most_volatile[0]}")
print(f"Monthly volatility: {highest_volatility:.2%}")

Most volatile month: 2010-06 for tsla
Monthly volatility: 28.84%
